# Lecture 11

We shall code the AR simulator for the probit model and then invert it using the inversion theorem.

Take a vector of systematic utilities

In [1]:
library(Matrix)
library(gurobi)
#seed = 777
#set.seed(seed)
U_y = c(0.4, 0.5, 0.2, 0.3, 0.1, 0)

Loading required package: slam


Simulate the market shares using the AR simulator

In [6]:
nbDraws = 1e+03
nbY = length(U_y)
rho = 0.5

Covar = rho * matrix(1, nbY, nbY) + (1 - rho) * diag(1, nbY)
E = eigen(Covar)
V = E$values
Q = E$vectors
SqrtCovar = Q %*% diag(sqrt(V)) %*% t(Q)
epsilon_iy = matrix(rnorm(nbDraws * nbY), ncol = nbY) %*% SqrtCovar
epsilon_iy2 = matrix(rnorm(nbDraws * nbY), ncol = nbY) %*% SqrtCovar

u_iy = t(t(epsilon_iy) + U_y)
ui = apply(X = u_iy, MARGIN = 1, FUN = max)
s_y = apply(X = u_iy - ui, MARGIN = 2, FUN = function(v) (length(which(v == 0))))/nbDraws

To invert the market share, simply run the optimal assignment problem

In [10]:
A1 = kronecker(matrix(1, 1, nbY), sparseMatrix(1:nbDraws, 1:nbDraws))
A2 = kronecker(sparseMatrix(1:nbY, 1:nbY), matrix(1, 1, nbDraws))
A = rbind2(A1, A2)
result = gurobi(list(A = A, obj = c(epsilon_iy2), modelsense = "max", rhs = c(rep(1/nbDraws, 
    nbDraws), s_y), sense = "="), params = list(OutputFlag = 0))
Uhat_y = -result$pi[(1 + nbDraws):(nbY + nbDraws)] + result$pi[(nbY + nbDraws)]

print("U_y (true and recovered)")
print(U_y)
print(Uhat_y)

[1] "U_y (true and recovered)"
[1] 0.4 0.5 0.2 0.3 0.1 0.0
[1] 0.37431123 0.56799142 0.27316773 0.37317525 0.06925918 0.00000000


## BLP Data

Let's take a look through the data of the Berry, Levinsohn and Pakes (1995), often referred to as BLP.

This data comes from the High Dimensional Metrics library (hdm) library. Instead of installing the package and retreiving the data (which takes a while because we need to compile these of the library), we will instead just load it up directly

In [1]:
load("BLPdata.Rdata")
BLPData
data = as.matrix(BLPData)

model.name,model.id,firm.id,cdid,id,price,mpd,air,mpg,space,hpwt,trend,share,outshr,y
AMGREM,23,15,1,129,-6.8256171,1.888146,0,1.697,1.1502,0.5289969,0,0.0010512928,0.8801063,0.82036558
AMHORN,24,15,1,130,-6.2453701,1.935989,0,1.740,1.2780,0.4943244,0,0.0006700762,0.8801063,0.36998106
AMJAVL,25,15,1,132,-4.6527775,1.716799,0,1.543,1.4592,0.4676134,0,0.0003405273,0.8801063,-0.30691499
AMMATA,26,15,1,134,-4.9219133,1.687871,0,1.517,1.6068,0.4265403,0,0.0005222419,0.8801063,0.12072050
AMAMBS,18,15,1,136,-2.8330245,1.504286,0,1.352,1.6458,0.4524887,0,0.0004424231,0.8801063,-0.04514379
BKSKYL,53,19,1,138,-4.6083331,1.726813,0,1.552,1.6224,0.4508706,0,0.0027560592,0.8801063,1.78414673
BKLSAB,45,19,1,141,-1.9046294,1.727926,0,1.553,1.7680,0.5640020,0,0.0026505939,0.8801063,1.74512865
BKCNTU,40,19,1,143,-0.4922837,2.194121,0,1.972,1.7680,0.7313675,0,0.0004873418,0.8801063,0.05155530
BKELCT,41,19,1,144,0.3743829,2.056154,0,1.848,1.8160,0.7190139,0,0.0024151269,0.8801063,1.65209674
BKRIVE,50,19,1,145,1.2089509,1.978269,0,1.778,1.7440,0.7283237,0,0.0005513978,0.8801063,0.17504615


In [2]:
BLPData$price

[1] -6.825617051 -6.245370137 -4.652777545 -4.921913347 -2.833024458
   [6] -4.608333100 -1.904629397 -0.492283718  0.374382949  1.208950850
  [11]  3.238580480  4.283024924  6.468210110  7.406481715 -6.600925693
  [16] -5.823147915 -5.151542977 -4.549073841 -3.326851619 -4.116975076
  [21] -3.778703471 -3.388580014 -1.554012113  1.900308875 -1.415740508
  [26] -0.188580014  0.362037270 -6.835493594 -5.712036804 -5.028086187
  [31] -4.904629397 -5.077468903 -3.618209644 -1.334259026 -7.023147915
  [36] -6.242901002 -5.163888656 -4.573765199 -3.642901002 -2.887345446
  [41] -2.055246681  1.312654554  6.557098998  9.999074307 -6.124382483
  [46] -4.902160261 -3.640431866 -2.235493594 -0.714505940 -4.356481248
  [51] -1.921913347  0.218827394  1.712654554 -5.855246681 -5.208333100
  [56] -5.010802236 -3.993518285 -5.618209644 -4.539197298 -4.237962730
  [61] -2.452777545 -1.366357792 -0.509567668 -4.119444211 -2.884876310
  [66] -5.724382483 -7.474999767 -6.526851619 -7.921913347 -6.623147915
  [71] -8.316975076 -6.341666434  1.354629863  1.618827394  3.986728628
  [76]  6.680555789 -7.062654088 -3.173765199 -2.884876310  4.275617517
  [81] -7.205863965 -3.428086187 -6.840431866 -6.452777545 -6.131789890
  [86] -4.726851619 -5.346604705 -7.205863965 -5.344135569 -4.356481248
  [91] -3.440431866 -1.526851619 -6.979122869 -6.342759233 -5.046108515
  [96] -5.101132439 -2.467161147 -2.292519999 -0.976730525 -0.062854927
 [101]  0.556762298  2.444322107  3.525661820  5.535231198  6.508915408
 [106] -6.833189855 -6.070031960 -5.455199424 -5.230319042 -3.718357319
 [111] -4.407352534 -4.096347750 -3.701610908 -2.170510429  1.329489571
 [116] -2.070031960 -0.122663539 -6.419314257 -5.971945836 -5.086778372
 [121] -5.251850142 -3.105917128 -1.964768802 -7.072424305 -6.510223348
 [126] -5.443237702 -5.352328611 -4.907352534 -3.995869281 -3.299697032
 [131] -2.455199424  0.901259906  4.453891485  5.707480001  8.908436939
 [136] -6.024577415 -4.959984113 -4.546108515 -2.687256841 -1.012615692
 [141] -4.426491290 -2.316443444 -0.261419520  1.016092442 -6.108309472
 [146] -5.278165932 -5.354720955 -3.474338180 -5.890606123 -5.029362104
 [151] -3.653764018 -4.971945836 -2.878644400 -1.787735310 -1.584386027
 [156] -4.117878850  1.528054164 -5.713572630 -7.034146793 -5.885821434
 [161] -6.957591769 -6.270988898 -4.835582199  3.231403446  3.580685743
 [166]  5.506523064 10.973030241 -6.859505644 -2.826012821 -2.778165932
 [171]  5.583078088 -6.270988898 -2.682472152 -6.543716171 -6.055677893
 [176] -5.615486506 -6.979122869 -5.184864496 -3.926491290 -3.172902774
 [181] -1.306874066 -7.036194295 -6.484392493 -5.254662763 -5.335743844
 [186] -1.714122223 -5.842500601 -4.876284385 -2.756915016 -1.874032133
 [191] -0.662320421 -0.002410511  1.837679579  2.878220120  4.815157057
 [196]  5.727319219 -7.060969070 -6.340248349 -5.497906007 -4.069978079
 [201] -4.448356457 -3.310969070 -2.608266367  0.930021921 -2.344752853
 [206] -0.506915016 -6.272680781 -6.121779880 -4.979887989 -5.304212313
 [211] -3.362770871 -2.261419520 -7.209617718 -6.569978079 -5.678086187
 [216] -5.545203304 -5.126284385 -3.639797898 -2.847005106  4.132724624
 [221]  5.071913813  8.472814714 -6.430338439 -5.407815916 -4.166824925
 [226] -2.840248349 -1.292951052 -5.813221322 -4.696104205 -2.772680781
 [231] -0.815473574  0.493084984 -6.250158259 -5.151059160 -5.401059160
 [236] -3.709617718 -6.173581682 -5.189347448 -5.241149250 -3.270428529
 [241] -2.385293394 -1.439347448 -6.817725826 -5.015924025 -5.592500601
 [246] -6.583491592 -6.042951052 -6.919077178 -5.916824925  4.378220120
 [251]  4.871463363 10.069661561 19.616958858 -5.880788889 -2.234392493
 [256] -1.290698799  6.704796696 -5.910068169 -3.101509610 -5.795203304
 [261] -6.290698799 -5.727635736 -4.646554655 -4.263671772 -3.655563664
 [266]  1.470562462 -1.986644745 -6.728965159 -6.033224794 -5.678255220
 [271] -5.570750149 -2.513955017 -5.554522968 -3.980486457 -2.927748120
 [276] -0.244178141  4.668600764  6.7172823